In [1]:
import tensorflow as tf
from utils import *

## Build the graph

Build the same graph for which the model variables are saved

In [10]:
def build_graph(X_train, Y_train, learning_rate = 0.009):
    (m, n_l, _, __) = X_train.shape
    n_y = Y_train.shape[1]
    
    X = tf.placeholder(tf.float32, shape=(None, n_l, 1, 1))
    Y = tf.placeholder(tf.float32, shape=(None, n_y))
    
    regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
    Z1 = tf.layers.conv2d(X, 8, (4,1), strides = [1,1], padding = 'SAME', kernel_regularizer = regularizer)
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1, ksize = [1,8,1,1], strides = [1,8,1,1], padding = 'SAME')
    Z2 = tf.layers.conv2d(P1, 16, (2, 1), strides = [1,1], padding = 'SAME', kernel_regularizer = regularizer)
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2, ksize = [1,4,1,1], strides = [1,4,1,1], padding = 'SAME')
    P2 = tf.contrib.layers.flatten(P2)
    Z3 = tf.contrib.layers.fully_connected(P2, 30, activation_fn=None, weights_regularizer = regularizer)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    return Z3, cost, optimizer, X, Y

## Load the data

In [14]:
config = tf.ConfigProto(device_count = {'GPU': 0})
sess = tf.InteractiveSession(config=config)

X_train = X_train_orig
X_test = X_test_orig
Y_train = convert_to_one_hot(Y_train_orig, classes)
Y_test = convert_to_one_hot(Y_test_orig, classes)
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1000
number of test examples = 60
X_train shape: (1000, 16000, 1, 1)
Y_train shape: (1000, 30)
X_test shape: (60, 16000, 1, 1)
Y_test shape: (60, 30)


## Load the saved model

In [15]:
# tf.reset_default_graph() # to be able to rerun the model without overwriting tf variables
Z3, cost, optimizer, X, Y = build_graph(X_train, Y_train)

### Using saver

In [16]:
saver = tf.train.Saver()
saver.restore(sess, "../saved_models/trained_model.ckpt")

INFO:tensorflow:Restoring parameters from ../saved_models/trained_model.ckpt


NotFoundError: Key fully_connected_3/weights/Adam_1 not found in checkpoint
	 [[Node: save_1/RestoreV2_79 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2_79/tensor_names, save_1/RestoreV2_79/shape_and_slices)]]

Caused by op u'save_1/RestoreV2_79', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/nipasuma/Projects/kaggle-tensorflow-speech-recognition/src/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/nipasuma/Projects/kaggle-tensorflow-speech-recognition/src/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/nipasuma/Projects/kaggle-tensorflow-speech-recognition/src/lib/python2.7/site-package

In [ ]:
print("W1 = " + str(W1.eval()[0,0,0]))

### Using meta_graph APIs

Requires explicit naming of variable. Did not work due to that

In [17]:
new_saver = tf.train.import_meta_graph('../saved_models/my-cnn-tf-model.meta')
new_saver.restore(sess, '../saved_models/my-cnn-tf-model.meta')

ValueError: At least two variables have the same name: fully_connected/biases